In [59]:
import os 

In [60]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [61]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")

In [62]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [63]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [64]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")

c:\Users\KEVIN\Desktop\25\Advanced-Langgraph\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [65]:
'''from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")'''
from langchain_groq import ChatGroq
import os
llm=ChatGroq(model_name="Gemma2-9b-It")

# SIMPLE AI ASSISTANT

In [66]:
while True:
    question=input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break

goodbye take care yourself


In [67]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [68]:

store={}

In [69]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [70]:
config = {"configurable": {"session_id": "firstchat"}}

In [71]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)


In [72]:
model_with_memory.invoke(("Hi! I'm kevin"),config=config).content


"Hi Kevin! 👋 \n\nIt's nice to meet you. What can I do for you today? 😊\n"

In [73]:
model_with_memory.invoke(("tell me what is my name?"),config=config).content


'Your name is Kevin!  You told me earlier. 😄 \n\nIs there anything else I can help you with?\n'

In [74]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm kevin", additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Kevin! 👋 \n\nIt's nice to meet you. What can I do for you today? 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 15, 'total_tokens': 41, 'completion_time': 0.047272727, 'prompt_time': 0.000210449, 'queue_time': 0.028817719, 'total_time': 0.047483176}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ce62df9f-6c3d-40d9-a4b3-6febf1799219-0', usage_metadata={'input_tokens': 15, 'output_tokens': 26, 'total_tokens': 41}), HumanMessage(content='tell me what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Kevin!  You told me earlier. 😄 \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'pr

# RAG with LCEL


In [75]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

### Reading the txt files from source directory

loader = DirectoryLoader('../data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings

'''from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
'''

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [76]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [77]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [78]:

question ="what is llama3? can you highlight 3 important points?"
print(retrieval_chain.invoke(question))

Llama 3 is an 8B parameter language model released in April 2024. 

Here are three important points:

* **Accessible:** Versions of Llama 3 have been made accessible outside of Meta.
* **Used in Services:** Llama 3 models are used in services on the Meta website. 
* **Parameter Size:** Llama 3 comes in an 8B parameter version. 





# Tools and Agents


## Wikipedia tool


In [79]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [80]:
api_wrapper = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=100)

In [81]:
tool1 = WikipediaQueryRun(api_wrapper=api_wrapper)

In [82]:
tool1.name

'wikipedia'

In [83]:
tool1.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [84]:
tool1.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [85]:
tool1.return_direct

False

In [86]:
print(tool1.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of 


In [87]:
tool1.run("William Rutto")

'Page: Ruto\nSummary: Ruto, or Rutto, is a surname of Kenyan origin. Notable people with the surname i'

# Youtube search tool

In [88]:
from langchain_community.tools import YouTubeSearchTool

In [89]:
tool2=YouTubeSearchTool()

In [90]:
tool2.name

'youtube_search'

In [91]:
tool2.description

'search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional'

In [92]:
tool2.run("nero knowledge")

"['https://www.youtube.com/watch?v=uGfOFyjQsNg&pp=ygUObmVybyBrbm93bGVkZ2U%3D', 'https://www.youtube.com/watch?v=uRgY58-JmjI&pp=ygUObmVybyBrbm93bGVkZ2U%3D']"

# Web search

In [93]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [94]:
tool3 = TavilySearchResults()

In [95]:
tool3.invoke({"query": "Is it correct for me to say humans we are magnetic beings"})

[{'url': 'https://www.discovermagazine.com/planet-earth/humans-can-sense-earths-magnetic-field-brain-imaging-study-says',
  'content': 'If human beings do possess a magnetic sense, daily experience suggests that it would be very weak or deeply subconscious. ... In normal life, when someone rotates their head - say, nodding up and down or turning the head from left to right - the direction of the geomagnetic field (which remains constant in space) will shift relative to'},
 {'url': 'https://www.ncesc.com/geographic-faq/are-human-beings-magnetic/',
  'content': 'However, there are some interesting magnetic properties exhibited by the human body, such as diamagnetism in response to strong magnetic fields. Is the human body slightly magnetic? Although there are some magnetic properties exhibited by the body, such as diamagnetism, humans are primarily composed of water and do not have a strong magnetic field. Yes, it is well demonstrated that normal physiological activities in the human bod

# Agent

In [107]:
from langchain import hub
from langchain.agents import AgentExecutor,create_openai_functions_agent

In [108]:
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

In [109]:
tavily_tool = TavilySearchResults()
wikipedia_tool = WikipediaQueryRun(api_wrapper=api_wrapper)
youtube_tool = YouTubeSearchTool()
tools = [tavily_tool]

In [110]:
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [111]:
print(agent_executor.invoke({"input": " who is william rutto"}))



> Entering new AgentExecutor chain...


> Finished chain.
{'input': ' who is william rutto', 'output': ''}
